In [ ]:
import os
import napari
import numpy as np
import pandas as pd
import re
import math
from scipy import stats
import skimage as ski
import dask.dataframe as dd
from tqdm import tqdm
from datetime import datetime
from bioio import BioImage
import bioio_tifffile

In [2]:
# Minimum number of pixels for a filament to be included in analysis
min_pixels = 3

# Minimum length (um) for a filament to be included in analysis.
# Can select 2D, 3D, or both.
min_length_2D = 0
min_length_3D = 0

In [ ]:
def parse_coordinates(coordinate_string, tuple_format=False):
	coords_str = coordinate_string[1:-1]
	coord_list = []
	for match in re.findall(r"\(.*?\)", coords_str):
		coordinate = match[1:-1]
		coord_list.append([float(coord) for coord in coordinate.split(",")])
	if tuple_format:
		return [tuple(coord) for coord in coord_list]
	return np.array(coord_list)

def parse_list(list_string):
	list_str = list_string[1:-1]
	return [int(item) for item in list_str.split(",")]

def get_distance(point_array, dims=3):
	if dims == 2:
		point_array = point_array[:, :2]
	diffs = np.diff(point_array, axis=0)
	distances = np.linalg.norm(diffs, axis=1)
	return np.sum(distances)

def get_radians_2D(p1, p2):
	radians = math.atan2(p2[1] - p1[1], p2[0] - p1[0])
	absolute_radians = np.mod(radians, np.pi)
	return absolute_radians


def get_average_radians(point_array, spacer=5):
	if spacer >= len(point_array):
		spacer = len(point_array) - 1
	xdiff = point_array[spacer:, 0] - point_array[:-spacer, 0]
	ydiff = point_array[spacer:, 1] - point_array[:-spacer, 1]
	radians = np.arctan2(ydiff, xdiff)
	mean_radians = stats.circmean(radians)
	absolute_radians = np.mod(mean_radians, np.pi)
	return absolute_radians

def compareAngles(a, b):
	delta = ((b - a + 180.0) % 360.0) - 180.0
	return np.abs(delta)

def get_branch_angles(df, vertices_col, angle_col, new_col):
	# Keep original order/id
	tmp = df.reset_index().rename(columns={"index": "_idx"})

	# 1) One vertex per row
	ex = tmp[["_idx", angle_col, vertices_col]].explode(vertices_col, ignore_index=True)

	# 2) Self-join on vertex to find neighbors sharing any vertex
	pairs = ex.merge(ex, on=vertices_col, how="inner", suffixes=("_src", "_tgt"))

	# 3) Drop self-pairs and (optionally) duplicate pairs that arise if two rows share multiple vertices
	pairs = pairs[pairs["_idx_src"] != pairs["_idx_tgt"]]
	pairs = pairs.drop_duplicates(["_idx_src", "_idx_tgt"])

	# 4) Vectorized angle diffs from src->tgt
	diffs = compareAngles(pairs[f"{angle_col}_src"].to_numpy(),
						   pairs[f"{angle_col}_tgt"].to_numpy())
	pairs = pairs.assign(diff=diffs)

	# 5) Keep strictly positive diffs, aggregate per source row as list
	pairs = pairs[pairs["diff"] > 0]
	agg = pairs.groupby("_idx_src")["diff"].apply(list).rename(new_col)

	# 6) Join back to original frame
	out = tmp.join(agg, on="_idx").drop(columns=["_idx"])

	# Preserve original index/columns order
	out.index = df.index
	return out

def get_residuals(point_array):
	NDModel = ski.measure.LineModelND()
	JustStartAndEnd = np.stack((point_array[0, :], point_array[-1, :]))
	NDModel.estimate(JustStartAndEnd)
	return np.mean(NDModel.residuals(point_array))

def get_curvature(coords, spacing=1):
	n = coords.shape[0]
	if n < (3 * spacing):
		return np.nan, np.nan
	xy = coords[:, :2]

	curv = []
	for i in range(spacing, n - spacing):
		e1, p, e2 = xy[i - spacing], xy[i], xy[i + spacing]
		v1, v2 = p - e1, e2 - p
		l1, l2, l3 = np.linalg.norm(v1), np.linalg.norm(v2), np.linalg.norm(e2 - e1)
		denom = l1 * l2 * l3
		if denom == 0:
			continue
		cross_z = v1[0] * v2[1] - v1[1] * v2[0]
		k = (abs(cross_z) / denom) * np.sign(cross_z)
		curv.append(k)

	if not curv:
		return np.nan, np.nan

	curv = np.array(curv)
	accumulated_curv = np.nanmean(np.abs(curv))
	net_curv = np.abs(np.nanmean(curv))
	return accumulated_curv, net_curv

def render_filaments(df, filename, viewer):
	onefile = df[df["Filename"] == filename]
	imp = BioImage(onefile["Filename"][0], reader=bioio_tifffile.Reader)
	image_data = imp.chunk[0, 0, :, :, :]
	scale = (imp.physical_pixel_sizes.Z, imp.physical_pixel_sizes.Y, imp.physical_pixel_sizes.X)
	viewer.add_image(image_chunk, scale=scale)
	coordlist = onefile["Coordinates (um)"].tolist()
	label_img = np.zeros(image_chunk.shape, dtype=np.uint16)
	coordlist = [np.round(coord[:, ::-1]/scale).astype(int) for coord in coordlist]
	for label_id, coord in enumerate(coordlist):
		for index in range(coord.shape[0]):
			label_img[(coord[index, 0], coord[index, 1], coord[index, 2])] = label_id + 1
	viewer.add_labels(label_img, scale=scale)

def transform_file(src_path, dst_path):
    total_bytes = os.path.getsize(src_path)
    with open(src_path, 'r', encoding='utf-8') as src, \
         open(dst_path, 'w', encoding='utf-8') as dst, \
        tqdm(total=total_bytes, unit='B', unit_scale=True, desc="Processing") as pbar:
        for line in src:
            pbar.update(len(line.encode('utf-8')))
            line = line.replace(',[' , ',"[').replace('],', ']",')
            dst.write(line)

In [7]:
# Input "Per_Filament_Coordinates.csv" filepath here
folder_path = r"D:\Data\Durham"
filename = "Per_Filament_Coordinates_Ianthe.csv"
savefilename = "Per_Filament_Processed.csv"

In [21]:
transform_file(os.path.join(folder_path, filename), os.path.join(folder_path, "Fixed_" + filename))

Processing: 100%|█████████▉| 36.3G/36.3G [04:57<00:00, 122MB/s] 


In [ ]:
datafile = os.path.join(folder_path, "Fixed_" + filename)
data = pd.read_csv(datafile, on_bad_lines="warn", chunksize=10000000)
buffer = None
chunksProcessed = 0
for chunk in data:
	if buffer is not None:
		chunk = pd.concat([buffer, chunk], ignore_index=True)
	# chunk["Filename"] = chunk["Filename"].astype("category")
	print (datetime.now().strftime("%H:%M:%S"),chunk.memory_usage(index=True).sum()/1024**3, "GB")
	last_filename = chunk["Filename"].iloc[-1]
	buffer = chunk[chunk["Filename"] == last_filename].copy()
	chunk = chunk.drop(chunk[chunk["Filename"] == last_filename].index)

	print(datetime.now().strftime("%H:%M:%S"), "Parsing Coordinates...")
	# Parses coordinate strings into numpy arrays
	chunk["Coordinates (um)"] = chunk["Coordinates (um)"].apply(parse_coordinates)
	chunk["Verticies"] = chunk["Verticies"].apply(parse_coordinates, tuple_format=True)
	# Parses vertex locations into lists
	chunk["Vertex Locations"] = chunk["Vertex Locations"].apply(parse_list)
	print(datetime.now().strftime("%H:%M:%S"), "Filtering for number of pixels...")
	# Getting number of pixels in filament
	chunk["Num. Pixels"] = chunk["Coordinates (um)"].apply(lambda coords: len(coords))
	# Filtering by minimum pixels
	chunk = chunk[chunk["Num. Pixels"] >= min_pixels]

	print(datetime.now().strftime("%H:%M:%S"), "Calculating lengths...")
	# Getting length of filament in 2D
	chunk["Length 2D (um)"] = chunk["Coordinates (um)"].apply(get_distance, dims=2)
	# Filtering by minimum lengths in 2D
	chunk = chunk[chunk["Length 2D (um)"] >= min_length_2D]
	# Getting length of filament in 3D
	chunk["Length 3D (um)"] = chunk["Coordinates (um)"].apply(get_distance, dims=3)
	# Filtering by minimum lengths in 3D
	chunk = chunk[chunk["Length 3D (um)"] >= min_length_3D]

	print(datetime.now().strftime("%H:%M:%S"), "Calculating angles...")
	# Gets angles by getting angle between points spaced by 'spacer' value and averaging them
	chunk["Average Angle (radians)"] = chunk["Coordinates (um)"].apply(get_average_radians, spacer=5)
	chunk["Average Angle (degrees)"] = np.degrees(chunk["Average Angle (radians)"])
	# Gets angle between the first and last point of the filament
	chunk["End to End Angle (radians)"] = chunk["Coordinates (um)"].apply(lambda coords: get_radians_2D(coords[0], coords[-1]))
	chunk["End to End Angle (degrees)"] = np.degrees(chunk["End to End Angle (radians)"])

	print(datetime.now().strftime("%H:%M:%S"), "Calculating branch angles...")
	# Gets branch angles by finding filaments that share verticies and comparing their average angles
	results = []
	for (fname, chan), subset in chunk.groupby(["Filename", "Channel"]):
		results.append(get_branch_angles(subset, "Verticies", "Average Angle (degrees)", "Branch Angles (degrees)"))
	chunk = pd.concat(results)

	print(datetime.now().strftime("%H:%M:%S"), "Calculating deviations...")
	#  Gets the mean distance from a straight line between the endpoints of the filament
	chunk["Deviation (um)"] = chunk["Coordinates (um)"].apply(get_residuals)

	print(datetime.now().strftime("%H:%M:%S"), "Calculating curvatures...")
	# Gets curvature values
	chunk[["Accumulated Curvature", "Net Curvature"]] = chunk["Coordinates (um)"].apply(lambda coords: get_curvature(coords, spacing=1)).apply(pd.Series)
	chunksProcessed += 1
	print(datetime.now().strftime("%H:%M:%S"), "Saving chunk", chunksProcessed)
	chunk.to_csv(os.path.join(folder_path, savefilename), mode='a', header=not os.path.exists(os.path.join(folder_path, savefilename)), index=False)
	

In [14]:
chunk

,Filename,Channel,Filament ID,Coordinates (um),Verticies,Vertex Locations,Num. Pixels,Length 2D (um),Length 3D (um),Average Angle (radians),Average Angle (degrees),End to End Angle (radians),End to End Angle (degrees)


In [109]:
data = databackup.copy()

This next cell saves the outputs

In [27]:
data.to_pickle(os.path.join(folder_path, "Per_Filament_Coordinates_Analyzed.pkl"))

This next cell can load in previously saved outputs

In [5]:
data = pd.read_pickle(os.path.join(folder_path, "Per_Filament_Coordinates_Analyzed.pkl"))
databackup = data.copy()

This next cell opens up napari

In [6]:
Viewer = napari.Viewer()

This next cell allows you to view a labelled image

In [7]:
render_filaments(data, data["Filename"][0], Viewer)